In [3]:
import numpy as np
import pandas as pd
import yfinance as yf
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout

In [4]:
stock_symbol = "GOOG"
start_date = "2010-01-01"
end_date = pd.Timestamp.today()
data = yf.download(stock_symbol, start = start_date, end = end_date)
data.reset_index(inplace = True)
data

[*********************100%***********************]  1 of 1 completed


,Date,Open,High,Low,Close,Adj Close,Volume
0,2010-01-04,15.615220,15.678981,15.547723,15.610239,15.571998,78541293
1,2010-01-05,15.620949,15.637387,15.480475,15.541497,15.503423,120638494
2,2010-01-06,15.588072,15.588072,15.102393,15.149715,15.112601,159744526
3,2010-01-07,15.178109,15.193053,14.760922,14.797037,14.760787,257533695
4,2010-01-08,14.744733,15.024933,14.672753,14.994298,14.957565,189680313
...,...,...,...,...,...,...,...
3713,2024-10-04,169.339996,169.550003,166.960007,168.559998,168.559998,11422100
3714,2024-10-07,169.139999,169.899994,164.130005,164.389999,164.389999,14034700
3715,2024-10-08,165.429993,166.100006,164.309998,165.699997,165.699997,11723900
3716,2024-10-09,164.854996,166.259995,161.119995,163.059998,163.059998,19649400


In [5]:
# Step 1: Prepare the training and testing data
data_train = pd.DataFrame({
    'Open': data['Open'][0: int(len(data) * 0.8)],
    'Close': data['Close'][0: int(len(data) * 0.8)]
})

data_test = pd.DataFrame({
    'Open': data['Open'][int(len(data) * 0.8): len(data)],
    'Close': data['Close'][int(len(data) * 0.8): len(data)]
})

print("Training Data shape:", data_train.shape)
print("Testing Data shape:", data_test.shape)

Training Data shape: (2974, 2)
Testing Data shape: (744, 2)


In [6]:
# Step 2: Scale the data using MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))
data_train_scaled = scaler.fit_transform(data_train)

In [7]:
# Step 3: Prepare the input (x) and output (y) for the LSTM model
x = []
y = []

# Create sequences of data
time_steps = 60  # Number of time steps to look back

for i in range(time_steps, data_train_scaled.shape[0]):
    x.append(data_train_scaled[i - time_steps:i])  # Previous 60 time steps
    y.append(data_train_scaled[i])  # Current Open and Close prices

x, y = np.array(x), np.array(y)

In [8]:
# Step 5: Build the LSTM model
model = Sequential()

model.add(LSTM(units=50, activation='relu', return_sequences=True, input_shape=(x.shape[1], x.shape[2])))
model.add(Dropout(0.2))

model.add(LSTM(units=60, activation='relu', return_sequences=True))
model.add(Dropout(0.3))

model.add(LSTM(units=80, activation='relu', return_sequences=True))
model.add(Dropout(0.4))

model.add(LSTM(units=120, activation='relu'))
model.add(Dropout(0.5))

# Output layer to predict both Open and Close prices
model.add(Dense(units=2))  # 2 units for Open and Close prices

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

d:\Shreya\Project\stock prediction\venv\lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [9]:
# Step 6: Train the model
model.fit(x, y, epochs=100, batch_size=32)  # Adjust epochs and batch size as needed


Epoch 1/100
92/92 ━━━━━━━━━━━━━━━━━━━━ 8s 52ms/step - loss: 0.0323
Epoch 2/100
92/92 ━━━━━━━━━━━━━━━━━━━━ 5s 54ms/step - loss: 0.0054
Epoch 3/100
92/92 ━━━━━━━━━━━━━━━━━━━━ 5s 51ms/step - loss: 0.0037
Epoch 4/100
92/92 ━━━━━━━━━━━━━━━━━━━━ 5s 53ms/step - loss: 0.0032
Epoch 5/100
92/92 ━━━━━━━━━━━━━━━━━━━━ 5s 52ms/step - loss: 0.0031
Epoch 6/100
92/92 ━━━━━━━━━━━━━━━━━━━━ 5s 52ms/step - loss: 0.0028
Epoch 7/100
92/92 ━━━━━━━━━━━━━━━━━━━━ 5s 54ms/step - loss: 0.0026
Epoch 8/100
92/92 ━━━━━━━━━━━━━━━━━━━━ 5s 51ms/step - loss: 0.0023
Epoch 9/100
92/92 ━━━━━━━━━━━━━━━━━━━━ 5s 55ms/step - loss: 0.0029
Epoch 10/100
92/92 ━━━━━━━━━━━━━━━━━━━━ 5s 53ms/step - loss: 0.0018
Epoch 11/100
92/92 ━━━━━━━━━━━━━━━━━━━━ 5s 54ms/step - loss: 0.0020
Epoch 12/100
92/92 ━━━━━━━━━━━━━━━━━━━━ 5s 53ms/step - loss: 0.0019
Epoch 13/100
92/92 ━━━━━━━━━━━━━━━━━━━━ 5s 53ms/step - loss: 0.0017
Epoch 14/100
92/92 ━━━━━━━━━━━━━━━━━━━━ 5s 51ms/step - loss: 0.0020
Epoch 15/100
92/92 ━━━━━━━━━━━━━━━━━━━━ 5s 51ms/step - lo

In [10]:
# Step 7: Prepare the test data (you can do this in a similar way)
data_test_scaled = scaler.transform(data_test)

x_test = []
for i in range(time_steps, data_test_scaled.shape[0]):
    x_test.append(data_test_scaled[i - time_steps:i])

x_test = np.array(x_test)
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], x_test.shape[2]))

In [11]:
# Step 8: Make predictions
predictions = model.predict(x_test)

22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 33ms/step


In [12]:
# Step 9: Inverse transform the predictions to get actual prices
predictions_inverse = scaler.inverse_transform(predictions)

In [13]:
# predictions_inverse will contain two columns: [Predicted Open, Predicted Close]
predicted_open_price = predictions_inverse[:, 0]
predicted_close_price = predictions_inverse[:, 1]

In [14]:
# Output the predicted prices
print("Predicted Open Prices:", predicted_open_price)
print("Predicted Close Prices:", predicted_close_price)

Predicted Open Prices: [140.70297  140.27325  139.57759  138.623    137.5402   136.51387
 135.69823  135.23654  135.21317  135.74797  136.73883  137.87112
 138.87471  139.54027  139.82574  139.80481  139.51598  139.01628
 138.43529  137.93892  137.56609  137.24739  136.89342  136.4464
 135.92065  135.45432  135.1794   135.1146   135.22498  135.4425
 135.64902  135.6637   135.3466   134.81865  134.28824  133.85905
 133.51422  133.20422  132.99728  132.98232  133.19788  133.62672
 134.22887  134.92612  135.64577  136.35095  137.00037  137.58327
 138.10458  138.52444  138.80627  138.9903   139.14488  139.24207
 139.20963  139.01929  138.62769  138.01735  137.25471  136.41959
 135.58102  134.841    134.28015  133.85545  133.38222  132.7152
 131.82056  130.59943  129.14548  127.59835  126.06308  124.69644
 123.69992  123.08831  122.68569  122.249146 121.69592  120.99924
 120.14516  119.27392  118.500404 117.937355 117.53448  117.131035
 116.602745 115.94305  115.056725 113.92693  112.69509 

In [15]:
# Save the trained model
model.save('stock_price_model.h5')